In [1]:
from pypokerengine.api.game import setup_config, start_poker
from pypokerengine.utils.card_utils import gen_cards
from pypokerengine.players import BasePokerPlayer
from baseline_players import RandomPlayer
from bots.honest_player import HonestPlayer
from bots.honest_modified_player import HonestModifiedPlayer
from bots.bot_denis_2 import BotDenisPlayer
from bots.bot1 import KoffaPlayer
# from bots.bot_denis_4 import 
from bots.forl_condition_bot_2 import ConditionPlayer
from bots.botDenis import GloomCha
from bots.forl_condition_bot import HonestStatPlayer
from bots.fish_player import FishPlayer
from bots.fold_player import FoldPlayer
from bots.bot_denis_3 import BotDenis3Player
from copy import deepcopy
import scipy.stats as sps
import keras.backend as K
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
%matplotlib inline

Using TensorFlow backend.


In [2]:
from bots.RLbot import RLPokerPlayer

In [3]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [2]:
N_RANDOM_PLAYERS = 0

config = setup_config(max_round=2, initial_stack=1500, small_blind_amount=15)
# config.register_player(name="foldman", algorithm=FoldPlayer())
config.register_player(name="honest", algorithm=HonestPlayer(200))
config.register_player(name="honest_modified", algorithm=HonestModifiedPlayer(200))
config.register_player(name="fishman", algorithm=FishPlayer())
config.register_player(name="botDenis1", algorithm=GloomCha())
config.register_player(name="botDenis2", algorithm=BotDenisPlayer())
config.register_player(name="conditionPlayer", algorithm=ConditionPlayer())
config.register_player(name="koffa", algorithm=KoffaPlayer())
config.register_player(name="botDenis3", algorithm=BotDenis3Player())
config.register_player(name="random", algorithm=RandomPlayer())

for i in range(N_RANDOM_PLAYERS):
    config.register_player(name="random" + str(i), algorithm=RandomPlayer())

In [3]:
%%time
from joblib import Parallel, delayed

def f(i,config):
    return start_poker(config, verbose=0)['players']

def get_all_results(config, n_games=1):
    players_stack_dict = {}
    for player_info in config.players_info:
        players_stack_dict[player_info['name']] = []
        
    with Parallel(n_jobs=4) as parallel:
#     for i in tqdm(range(n_games)):
        data = parallel(delayed(f)(i, config) for i in range(n_games))
    
    for i in tqdm(range(n_games)):
        for player_info in data[i]:
            players_stack_dict[player_info['name']].append(player_info['stack'])
            
    return pd.DataFrame(players_stack_dict)

res = get_all_results(config, n_games=1)

{'action': 'fold', 'amount': 0, 'player_uuid': 'icwvkyqtoteqgbyrubdtai'}
{   'action_histories': {   'preflop': [   {   'action': 'SMALLBLIND',
                                               'add_amount': 15,
                                               'amount': 15,
                                               'uuid': 'ejjipazgbubgrgyhpgctrl'},
                                           {   'action': 'BIGBLIND',
                                               'add_amount': 15,
                                               'amount': 30,
                                               'uuid': 'zkswpvgvkkpcrpsxtkiwrc'},
                                           {   'action': 'FOLD',
                                               'uuid': 'icwvkyqtoteqgbyrubdtai'}]},
    'big_blind_pos': 2,
    'community_card': [],
    'dealer_btn': 0,
    'next_player': 3,
    'pot': {'main': {'amount': 45}, 'side': []},
    'round_count': 1,
    'seats': [   {   'name': 'honest',
                   

                                               'add_amount': 15,
                                               'amount': 15,
                                               'uuid': 'ejjipazgbubgrgyhpgctrl'},
                                           {   'action': 'BIGBLIND',
                                               'add_amount': 15,
                                               'amount': 30,
                                               'uuid': 'zkswpvgvkkpcrpsxtkiwrc'},
                                           {   'action': 'FOLD',
                                               'uuid': 'icwvkyqtoteqgbyrubdtai'},
                                           {   'action': 'CALL',
                                               'amount': 30,
                                               'paid': 30,
                                               'uuid': 'gkmfhjhjvfnlfoppwjxwjd'}]},
    'big_blind_pos': 2,
    'community_card': [],
    'dealer_btn': 0,
    'next_player': 4,
    'p

                 {   'name': 'botDenis2',
                     'stack': 1470,
                     'state': 'participating',
                     'uuid': 'gkmfhjhjvfnlfoppwjxwjd'},
                 {   'name': 'conditionPlayer',
                     'stack': 1425,
                     'state': 'participating',
                     'uuid': 'vxzmdimeqqsaxtdfzrvypd'},
                 {   'name': 'koffa',
                     'stack': 1500,
                     'state': 'participating',
                     'uuid': 'ruhinopjfxnkuixgequfny'},
                 {   'name': 'botDenis3',
                     'stack': 1500,
                     'state': 'participating',
                     'uuid': 'cqlfdimzkfvbhnqogxgzdk'},
                 {   'name': 'random',
                     'stack': 1500,
                     'state': 'participating',
                     'uuid': 'rbcnunxyxywwcryzgloyuo'}],
    'small_blind_amount': 15,
    'small_blind_pos': 1,
    'street': 'preflop'}
{'action': 'fo

                                               'amount': 30,
                                               'uuid': 'zkswpvgvkkpcrpsxtkiwrc'},
                                           {   'action': 'FOLD',
                                               'uuid': 'icwvkyqtoteqgbyrubdtai'},
                                           {   'action': 'CALL',
                                               'amount': 30,
                                               'paid': 30,
                                               'uuid': 'gkmfhjhjvfnlfoppwjxwjd'},
                                           {   'action': 'RAISE',
                                               'add_amount': 45,
                                               'amount': 75,
                                               'paid': 75,
                                               'uuid': 'vxzmdimeqqsaxtdfzrvypd'},
                                           {   'action': 'FOLD',
                                             

                                               'uuid': 'ruhinopjfxnkuixgequfny'},
                                           {   'action': 'CALL',
                                               'amount': 75,
                                               'paid': 75,
                                               'uuid': 'cqlfdimzkfvbhnqogxgzdk'},
                                           {   'action': 'RAISE',
                                               'add_amount': 876,
                                               'amount': 951,
                                               'paid': 951,
                                               'uuid': 'rbcnunxyxywwcryzgloyuo'}]},
    'big_blind_pos': 2,
    'community_card': [],
    'dealer_btn': 0,
    'next_player': 8,
    'pot': {'main': {'amount': 1176}, 'side': []},
    'round_count': 1,
    'seats': [   {   'name': 'honest',
                     'stack': 1500,
                     'state': 'participating',
                     'uui

...

                     'uuid': 'ruhinopjfxnkuixgequfny'},
                 {   'name': 'botDenis3',
                     'stack': 3196,
                     'state': 'folded',
                     'uuid': 'cqlfdimzkfvbhnqogxgzdk'},
                 {   'name': 'random',
                     'stack': 40,
                     'state': 'folded',
                     'uuid': 'rbcnunxyxywwcryzgloyuo'}],
    'small_blind_amount': 15,
    'small_blind_pos': 2,
    'street': 'river'}
{   'action_histories': {   'flop': [   {   'action': 'CALL',
                                            'amount': 0,
                                            'paid': 0,
                                            'uuid': 'zkswpvgvkkpcrpsxtkiwrc'},
                                        {   'action': 'FOLD',
                                            'uuid': 'cqlfdimzkfvbhnqogxgzdk'},
                                        {   'action': 'CALL',
                                            'amount': 0,
        

                                               'amount': 30,
                                               'paid': 30,
                                               'uuid': 'rdykdhdcenqicbfusskjsn'},
                                           {   'action': 'FOLD',
                                               'uuid': 'ejjipazgbubgrgyhpgctrl'},
                                           {   'action': 'CALL',
                                               'amount': 30,
                                               'paid': 15,
                                               'uuid': 'zkswpvgvkkpcrpsxtkiwrc'},
                                           {   'action': 'FOLD',
                                               'uuid': 'icwvkyqtoteqgbyrubdtai'}],
                            'river': [   {   'action': 'CALL',
                                             'amount': 0,
                                             'paid': 0,
                                             'uuid': 'zkswp

                 {   'name': 'botDenis2',
                     'stack': 549,
                     'state': 'folded',
                     'uuid': 'gkmfhjhjvfnlfoppwjxwjd'},
                 {   'name': 'conditionPlayer',
                     'stack': 1425,
                     'state': 'folded',
                     'uuid': 'vxzmdimeqqsaxtdfzrvypd'},
                 {   'name': 'koffa',
                     'stack': 1500,
                     'state': 'folded',
                     'uuid': 'ruhinopjfxnkuixgequfny'},
                 {   'name': 'botDenis3',
                     'stack': 3196,
                     'state': 'folded',
                     'uuid': 'cqlfdimzkfvbhnqogxgzdk'},
                 {   'name': 'random',
                     'stack': 40,
                     'state': 'folded',
                     'uuid': 'rbcnunxyxywwcryzgloyuo'}],
    'small_blind_amount': 15,
    'small_blind_pos': 2,
    'street': 'river'}


100%|██████████| 1/1 [00:00<00:00, 4447.83it/s]

CPU times: user 3.48 s, sys: 2.73 s, total: 6.22 s
Wall time: 42.7 s


In [12]:
res.describe()

,botDenis1,botDenis2,conditionPlayer,fishman,foldman,honest,honest_modified,koffa,random
count,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.00000,30.000000
mean,2220.100000,3265.233333,1037.366667,1587.183333,1226.500000,879.976667,1285.400000,1510.60000,480.833333
std,2342.734967,3374.607182,1933.654562,3000.216717,111.395149,1758.441862,924.871794,352.56375,1359.559794
min,0.000000,0.000000,0.000000,0.000000,825.000000,0.000000,0.000000,1035.00000,0.000000
25%,1143.750000,0.000000,0.000000,0.000000,1185.000000,0.000000,213.750000,1165.87500,0.000000
50%,1342.500000,2507.500000,0.000000,0.000000,1230.000000,0.000000,1443.500000,1502.10000,0.000000
75%,2775.000000,5761.500000,714.000000,1648.125000,1286.250000,1032.375000,1771.250000,1781.25000,0.000000
max,9055.000000,10391.000000,5865.000000,11336.400000,1455.000000,6985.000000,3105.000000,2364.30000,5250.000000


In [ ]:
cnt = 0
class PrintPlayer(BasePokerPlayer):  # Do not forget to make parent class as "BasePokerPlayer"
    stats = PlayerStats()
    cnt = 0
    #  we define the logic to make an action through this method. (so this method would be the core of your AI)
    def declare_action(self, valid_actions, hole_card, round_state):
        # valid_actions format => [raise_action_info, call_action_info, fold_action_info]
        self.stats.update(hole_card, round_state)
        print(self.stats.calc_params(hole_card, round_state))
        call_action_info = valid_actions[1]
        action, amount = call_action_info["action"], call_action_info["amount"]
        return action, amount   # action returned here is sent to the poker engine

    def receive_game_start_message(self, game_info):
        self.stats.init_player_names(game_info)
        pass

    def receive_round_start_message(self, round_count, hole_card, seats):
        pass

    def receive_street_start_message(self, street, round_state):
        pass

    def receive_game_update_message(self, action, round_state):
        pass

    def receive_round_result_message(self, winners, hand_info, round_state):
        pass

In [4]:
N = 100

RLBotPark = []
DefaultBotPark = [RandomPlayer(), HonestPlayer(), FishPlayer(), 
           BotDenisPlayer(), HonestModifiedPlayer(), ConditionPlayer(), 
           GloomCha(), KoffaPlayer(), HonestStatPlayer(),
          ]

rl_score = []


for i in range(N):
    in_model_file = None if i == 0 else 'model_{}.h5'.format(i)
    out_model_file = 'model_{}.h5'.format(i + 1)
    config = setup_config(max_round=50, initial_stack=1500, small_blind_amount=15)
    humans = list(np.random.choice(DefaultBotPark, replace=True, size=4 + max(0, 4 - len(RLBotPark))))
    rlbots = list(np.random.choice(RLBotPark, replace=True, size=min(4, len(RLBotPark)))) if len(RLBotPark) > 0 else []
    print(humans, rlbots)
    players = humans + rlbots + [None]
    np.random.shuffle(players)
    print(players)
    for player in players:
        if player is None:
            RLP =  RLPokerPlayer(
                study_mode=True,
                model_file=in_model_file,
                gammaReward=0.1,
                alphaUpdateNet=0.1,
                epsilonRandom=0.05,
                players=players,
                max_history_len=1000,
                new_model_file=out_model_file,
            )
            config.register_player(name='RL',algorithm=RLP)    
        else:
            config.register_player(name=str(player), algorithm=player)
    info = start_poker(config, verbose=0)
    RLP.study_mode = False
    for player_info in info:
        if player_info['name'] == 'RL':
            stack = player_info['stack']
            rl_score.append(stack)
            print(stack)
            break

[<bots.bot1.KoffaPlayer object at 0x11c640b00>, <baseline_players.RandomPlayer object at 0x11c640978>, <bots.botDenis.GloomCha object at 0x11c640ac8>, <bots.botDenis.GloomCha object at 0x11c640ac8>, <bots.honest_player.HonestPlayer object at 0x11c6409b0>, <bots.bot1.KoffaPlayer object at 0x11c640b00>, <bots.bot1.KoffaPlayer object at 0x11c640b00>, <bots.bot_denis_2.BotDenisPlayer object at 0x11c640a20>] []
[<bots.botDenis.GloomCha object at 0x11c640ac8>, <bots.bot1.KoffaPlayer object at 0x11c640b00>, None, <bots.bot1.KoffaPlayer object at 0x11c640b00>, <bots.botDenis.GloomCha object at 0x11c640ac8>, <baseline_players.RandomPlayer object at 0x11c640978>, <bots.bot_denis_2.BotDenisPlayer object at 0x11c640a20>, <bots.bot1.KoffaPlayer object at 0x11c640b00>, <bots.honest_player.HonestPlayer object at 0x11c6409b0>]


NameError: name 'RLPokerPlayer' is not defined

In [22]:
[1,2] + []

[1, 2]

In [32]:
str(HonestPlayer())

'<bots.honest_player.HonestPlayer object at 0x11c2326d8>'